# Import

In [286]:
import pandas as pd

In [287]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [288]:
train_df = pd.read_csv("../../data/train.csv")
test_df = pd.read_csv("../../data/test.csv")  
submission = pd.read_csv("../../data/sample_submission.csv")  

In [289]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54609 non-null  object
 1   분류      54609 non-null  object
 2   제목      54609 non-null  object
 3   키워드     54609 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [290]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
dtypes: object(3)
memory usage: 548.7+ KB


In [291]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   분류      23405 non-null  object
dtypes: object(2)
memory usage: 365.8+ KB


In [292]:
len(train_df['분류'].value_counts())

56

타겟변수 -> '분류'이고 종류 56가지

In [293]:
test_df.head()

,ID,제목,키워드
0,TEST_00000,[부고] 김태수씨 별세 외,"김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,이노코리아,대표,희정,한성대..."
1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력","신규,확진,나흘,세자릿수,방역당국,핼러윈,풍선,효과,차단,총력,감염증,신종,코로나바..."
2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...","전해철,장관,재정,분권,강화,지방자치,2.0,시대,마중물,마련,장관,전해철,행정안전..."
3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사","용인시,구인,장애인,구직,만남,채용,행사,노호근,용인특례시,장애인,취업,지원,대회의..."
4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,"지자체,경기,북동부,지역,산업단지,혁신단위,설정,전략,지역,연계,특성,제시,경기도경..."


영어 번역 내용 제거

In [294]:
# '키워드' 열에서 '기사'와 '구글' 두 단어를 모두 포함하는 행을 찾는 조건
condition = train_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_df = train_df[condition].copy()

In [295]:
filtered_df.iloc[0]['키워드']

'이상일,용인,특례,시장,참석,민주평통,경기,지역,의장,표창,수여식,시장,민주평통,평화,수호,선도,활동,감사,강조,이상일,용인,특례,시장,에이스홀,시청,민주평화통일자문회,수여식,경기,지역,의장,표창,참석,관계자들,격려,자리,시장,홍승표,민주평통,경기,지역,회의,부의장,용인시협의회장,추상구,민주평통,용인시,협의회장,자문위원,민주평통,경기,지역,자문,위원,400여명,참석,시장,민주평화통일자문회,경기지역회의,안보태세,수호,한반도,평화,선도,활동,감사,시장,북한,도발,주시,차례,특강,정세,한반도,안보,강화,강조,내년,민주평통,활동,응원,차원,지원,수여식,이날,표창,임명숙,자문위원,장석순,용인시협의회,자문,위원,자치분권,정지혜,용인시,자치,분권,민간협력팀장,62명,유공자,의장,표창,용인시협의회,평화통일포럼,통일공감사업,진행,홍보,평화,통일,정책,노력,유공,기관,표창,대표,이소연,뉴코리아여성연합,통일,페이스,주제,특강,진행,기사,구글,번역,번역,영문,기사,전문,구글번역,이해도,노력,영문,번역,오류,전제,following,full,text,English,article,translated,Google,Translate,Google,Translate,hard,Mayor,Lee,emphasized,grateful,Break,News,Southern,Gyeonggi,Yongin,Reporter,Lee,Gwi-seon,Yongin,Special,Mayor,Lee,Sang-il,attended,Gyeonggi,Region,Chairman,Award,Award,Ceremony,Democratic,Peaceful,Unification,Advisory,Council,held,City,Hall,Ace,Hall,20t,encouraged,people,attended,event,including,Mayor,Lee,Hong,Seung-pyo,vice-chairman,Gyeonggi,Regional,Council,Democratic,People,Unification,Party,Sang-gu,Cho

In [296]:
len(filtered_df['분류'].value_counts())

47

In [297]:
# '기사, 구글' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['기사', '구글']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    # '기사' 앞에 있는 ','도 함께 지우기 위해 수정
    article_index = text.find('기사')
    if article_index > 0 and text[article_index - 1] == ',':
        article_index -= 1
    return text[:article_index].strip()

# 각 행의 '키워드' 값에서 '기사' 단어부터 마지막 단어까지 삭제
filtered_df['키워드'] = filtered_df['키워드'].apply(remove_keywords)

In [298]:
filtered_df.iloc[25]['키워드']

'용인시,수지구보건소,홈스쿨,수지,예비,부모,운영,용인시,수지구보건소,온라인,태교,교육,프로그램,수지,예비,부모,홈스쿨,운영,수지구보건소,시간,제약,인원,제한,교육,단점,보완,예비부모들,정보,제공,온라인,태교,교육,프로그램,진행,13일,홈스쿨,수지,예비,부모,아빠,예비,부모,교실,DIY,태교,교실,아기,안전,교실,모유,수유,교실,아빠태교교실,실전,아기돌,교실,순산,요가,중기,순산,요가,말기,강좌,구성,1인,최대,수강,참가,구민,수지구,보건소,홈페이지,신청,네이버,밴드,가입,프로그램,접속,강의,기간,참여,신청,일정,홈페이지,공지,사항,참고,4주,접속,수강,수지구,보건소,관계자,산모,임신,전후,배우자,도움,교육내용,단계별,구성,예비,아빠,엄마,관심,참여'

형태소분석기 : okt

In [299]:
from konlpy.tag import Okt

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 제외할 품사 목록 정의
exclude_pos = [
    'Suffix',       # 접미사
    'Determiner',   # 관형사
    'Adverb',       # 부사
    'Conjunction',  # 접속사
    'Josa',         # 조사
    'PreEomi',      # 선어말 어미
    'Eomi',         # 어미
    'Punctuation',  # 구두점
    # 'Foreign',      # 외국어
    # 'Alpha',        # 알파벳
    # 'Number',       # 숫자
    'Unknown'       # 알 수 없는 품사
]

# '제목' 열에서 일반명사와 고유명사만 추출하여 '키워드_추출' 열에 저장하는 함수
def extract_nouns_and_proper_nouns_from_title(title):
    words = okt.pos(title)
    nouns = [word for word, pos in words if pos in ['Noun'] and word not in exclude_pos]
    return ', '.join(nouns)

# '제목' 열에 함수 적용하여 '키워드_추출' 열 생성
filtered_df['키워드_추출okt'] = filtered_df['제목'].apply(extract_nouns_and_proper_nouns_from_title)

# 결과 출력
print(filtered_df[['제목', '키워드_추출okt']].head(10))

                                           제목  \
38          이상일 용인특례시장, 민주평통 경기지역 의장표창 수여식 참석   
75               용인특례시, 식품산업 박람회 열어 우수 농산물 홍보   
86                 용인특례시, 경기도 특별조정교부금 70억원 확보   
109        용인특례시, 모현 공동주택 현장 ‘찾아가는 행복소통시장실’가동   
137           용인특례시, ‘백옥쌀’ 활용한 제빵용 ‘백옥쌀가루’ 출시   
160            용인시, 농산물 가공 장비 활용도를 높이는 매뉴얼 발간   
164                 용인시 다함께돌봄센터 12호점과 13호점 개소   
165                 용인시, 지역일자리 사업 참여자 300명 모집   
168                       용인시, ‘사랑의 감자 심기’ 행사   
185  경기도의회 김재균 의원, ‘ 남사진위 IC ’ 북평택 진출입로 개선 촉구   

                                       키워드_추출okt  
38     이상일, 용인, 특례시, 민주, 평통, 기지역, 의장, 표창, 수여, 참석  
75             용인, 특례시, 식품, 산업, 박람회, 우수, 농산물, 홍보  
86                  용인, 특례시, 경기도, 특별, 정교, 부금, 확보  
109      용인, 특례시, 모현, 공동, 주택, 현장, 행복, 소통시, 실, 가동  
137     용인, 특례시, 백옥, 쌀, 활용, 제빵, 용, 백옥, 쌀, 가루, 출시  
160                용인시, 농산물, 가공, 장비, 활용, 매뉴얼, 발간  
164                              용인시, 센터, 호점, 호점  
165                 용인시, 지역, 일자리, 사업, 참여자, 명, 모집  
1

형태소분석기 : kkma

In [300]:
from konlpy.tag import Kkma

# Kkma 형태소 분석기 객체 생성
kkma = Kkma()

# 제외할 품사 목록 정의
exclude_pos = [
    'Suffix',       # 접미사
    'Determiner',   # 관형사
    'Adverb',       # 부사
    'Conjunction',  # 접속사
    'Josa',         # 조사
    'PreEomi',      # 선어말 어미
    'Eomi',         # 어미
    'Punctuation',  # 구두점
    # 'Foreign',      # 외국어
    # 'Alpha',        # 알파벳
    # 'Number',       # 숫자
    'Unknown'       # 알 수 없는 품사
]

# '제목' 열에서 일반명사와 고유명사만 추출하여 '키워드_추출' 열에 저장하는 함수
def extract_nouns_and_proper_nouns_from_title(title):
    words = kkma.pos(title)
    nouns = [word for word, pos in words if pos in ['NNG', 'NNP'] and word not in exclude_pos]
    return ', '.join(nouns)

# '제목' 열에 함수 적용하여 '키워드_추출' 열 생성
filtered_df['키워드_추출kkma'] = filtered_df['제목'].apply(extract_nouns_and_proper_nouns_from_title)

# 결과 출력
print(filtered_df[['제목', '키워드_추출kkma']].head(10))

                                           제목  \
38          이상일 용인특례시장, 민주평통 경기지역 의장표창 수여식 참석   
75               용인특례시, 식품산업 박람회 열어 우수 농산물 홍보   
86                 용인특례시, 경기도 특별조정교부금 70억원 확보   
109        용인특례시, 모현 공동주택 현장 ‘찾아가는 행복소통시장실’가동   
137           용인특례시, ‘백옥쌀’ 활용한 제빵용 ‘백옥쌀가루’ 출시   
160            용인시, 농산물 가공 장비 활용도를 높이는 매뉴얼 발간   
164                 용인시 다함께돌봄센터 12호점과 13호점 개소   
165                 용인시, 지역일자리 사업 참여자 300명 모집   
168                       용인시, ‘사랑의 감자 심기’ 행사   
185  경기도의회 김재균 의원, ‘ 남사진위 IC ’ 북평택 진출입로 개선 촉구   

                                            키워드_추출kkma  
38   이상, 일, 용인, 특례, 시장, 민주, 평, 통, 경기, 지역, 의장, 표창, 수...  
75                 용인, 특례, 시, 식품, 산업, 박람회, 우수, 농산물, 홍보  
86                     용인, 특례, 시, 경기도, 특별, 조정, 교부금, 확보  
109     용인, 특례, 시, 모, 현, 공동, 주택, 현장, 행복, 소통, 시장, 실, 가동  
137             용인, 특례, 시, 백옥, 쌀, 활용, 빵, 백옥, 쌀, 가루, 출시  
160                     용인시, 농산물, 가공, 장비, 활용도, 매뉴얼, 발간  
164                                   용인시, 봄, 센터, 점, 점  
165 

형태소분석기 : 단순제거

In [301]:
import re

# '제목' 열에서 어절 단위로 자르고 불필요한 공백이나 쉼표를 제거하는 함수
def extract_words_from_title(title):
    words = title.split()
    cleaned_words = [re.sub(r'[^\w\s]', '', word) for word in words if word.strip()]
    return ', '.join(cleaned_words)

# '제목' 열에 함수 적용하여 '키워드_추출simple' 열 생성
filtered_df['키워드_추출simple'] = filtered_df['제목'].apply(extract_words_from_title)

3가지 방법에 대한 결과 비교

In [315]:
def print_filtered_df_info(index):
    try:
        title = filtered_df.iloc[index]['제목']
        category = filtered_df.iloc[index]['분류']
        keywords = filtered_df.iloc[index]['키워드']
        keywords_okt = filtered_df.iloc[index]['키워드_추출okt']
        keywords_kkma = filtered_df.iloc[index]['키워드_추출kkma']
        keywords_simple = filtered_df.iloc[index]['키워드_추출simple']
        
        # '제목'의 마지막 단어 추출
        last_word_in_title = title.split()[-1]
        
        # '키워드'에서 '제목'의 마지막 단어와 일치하는 단어까지 출력
        keywords_list = keywords.split(',')
        if last_word_in_title in keywords_list:
            last_word_index = keywords_list.index(last_word_in_title) + 1
            keywords_limited = ','.join(keywords_list[:last_word_index])
        else:
            keywords_limited = ','.join(keywords_list[:10])  # 최대 10개의 단어만 출력
        
        print(f"제목: {title}")
        print(f"분류: {category}")
        print(f"키워드: {keywords_limited}")
        print(f"키워드_추출okt: {keywords_okt}")
        print(f"키워드_추출kkma: {keywords_kkma}")
        print(f"키워드_추출simple: {keywords_simple}")  # 새로 생성한 변수 출력
    except IndexError:
        print(f"인덱스 {index}가 데이터프레임의 범위를 벗어났습니다.")
    except KeyError as e:
        print(f"열 이름 오류: {e}")

In [316]:
print_filtered_df_info(0)

제목: 이상일 용인특례시장, 민주평통 경기지역 의장표창 수여식 참석
분류: 정치:정치일반
키워드: 이상일,용인,특례,시장,참석
키워드_추출okt: 이상일, 용인, 특례시, 민주, 평통, 기지역, 의장, 표창, 수여, 참석
키워드_추출kkma: 이상, 일, 용인, 특례, 시장, 민주, 평, 통, 경기, 지역, 의장, 표창, 수여식, 참석
키워드_추출simple: 이상일, 용인특례시장, 민주평통, 경기지역, 의장표창, 수여식, 참석


In [317]:
print_filtered_df_info(10)

제목: GH, '경기용인 플랫폼시티 랜드마크 조성' 아이디어 공모전 개최
분류: 지역
키워드: GH,경기용,조성,플랫폼,시티,랜드마크,개최
키워드_추출okt: 경기, 용인, 플랫폼, 시티, 랜드마크, 조성, 아이디어, 공모전, 개최
키워드_추출kkma: 경기, 용인, 플랫폼, 시티, 랜드, 마크, 조성, 아이디어, 공모전, 개최
키워드_추출simple: GH, 경기용인, 플랫폼시티, 랜드마크, 조성, 아이디어, 공모전, 개최


In [318]:
print_filtered_df_info(30)

제목: 용인시 “여성의 경제 참여 증진과 성평등 도시 구현에 힘 모아 달라”
분류: 사회:사회일반
키워드: 용인시,여성,증진,경제,참여,도시,백군기,용인,시장,경기도여성단체협의회
키워드_추출okt: 용인시, 여성, 경제, 참여, 증진, 평등, 도시, 구현, 힘, 달라
키워드_추출kkma: 용인시, 여성, 경제, 참여, 증진, 성, 평등, 도시, 구현, 힘
키워드_추출simple: 용인시, 여성의, 경제, 참여, 증진과, 성평등, 도시, 구현에, 힘, 모아, 달라


In [319]:
print_filtered_df_info(50)

제목: 용인시의회 김상수 의원, '무장애 도시 조성 조례안' 가결
분류: 지역
키워드: 의원,용인시의회,김상수,도시,조성,조례안,가결
키워드_추출okt: 용인시, 의회, 김상수, 의원, 무장애, 도시, 조성, 조례, 안, 가결
키워드_추출kkma: 용인, 시의회, 김, 상수, 의원, 무, 장애, 도시, 조성, 조례, 안, 가결
키워드_추출simple: 용인시의회, 김상수, 의원, 무장애, 도시, 조성, 조례안, 가결


okt, kkma, simple 3가지 형태소 분석 방법중 어떤 방법이 좋은지 명확하지 않아서  
일단은 okt가 좋다고 가정하고 okt로 추출한 키워드와 원본데이터에서의 키워드를 비교하여  
차이가 나는 단어를 출력하여 확인하는 코드가 아래 내용

In [320]:
def print_different_keywords(index):
    try:
        title = filtered_df.iloc[index]['제목']
        keywords = filtered_df.iloc[index]['키워드']
        keywords_okt = filtered_df.iloc[index]['키워드_추출okt']
        
        # '제목'의 마지막 단어 추출
        last_word_in_title = title.split()[-1]
        
        # '키워드'에서 '제목'의 마지막 단어와 일치하는 단어까지 출력
        keywords_list = keywords.split(',')
        if last_word_in_title in keywords_list:
            last_word_index = keywords_list.index(last_word_in_title) + 1
            keywords_limited = ','.join(keywords_list[:last_word_index])
        else:
            keywords_limited = ','.join(keywords_list[:10])  # 최대 10개의 단어만 출력
        
        # 공백 제거
        keywords_limited_set = set(word.strip() for word in keywords_limited.split(','))
        keywords_okt_set = set(word.strip() for word in keywords_okt.split(','))
        
        # keywords_limited와 keywords_okt가 다른 키워드를 출력하는지 확인
        different_keywords = keywords_limited_set.symmetric_difference(keywords_okt_set)
        
        # 다른 키워드가 각각 어디에 속하는지 출력
        different_keywords_in_limited = keywords_limited_set - keywords_okt_set
        different_keywords_in_okt = keywords_okt_set - keywords_limited_set
        
        print(f"제목: {title}")
        print(f"키워드: {keywords_limited}")
        print(f"키워드_추출okt: {keywords_okt}")
        print(f"다른 키워드 (키워드에만 있음): {', '.join(different_keywords_in_limited)}")
        print(f"다른 키워드 (키워드_추출okt에만 있음): {', '.join(different_keywords_in_okt)}")
    except IndexError:
        print(f"인덱스 {index}가 데이터프레임의 범위를 벗어났습니다.")
    except KeyError as e:
        print(f"열 이름 오류: {e}")

In [329]:
print_different_keywords(0)

제목: 이상일 용인특례시장, 민주평통 경기지역 의장표창 수여식 참석
키워드: 이상일,용인,특례,시장,참석
키워드_추출okt: 이상일, 용인, 특례시, 민주, 평통, 기지역, 의장, 표창, 수여, 참석
다른 키워드 (키워드에만 있음): 시장, 특례
다른 키워드 (키워드_추출okt에만 있음): 의장, 기지역, 표창, 평통, 특례시, 민주, 수여


In [325]:
print_different_keywords(10)

제목: GH, '경기용인 플랫폼시티 랜드마크 조성' 아이디어 공모전 개최
키워드: GH,경기용,조성,플랫폼,시티,랜드마크,개최
키워드_추출okt: 경기, 용인, 플랫폼, 시티, 랜드마크, 조성, 아이디어, 공모전, 개최
다른 키워드 (키워드에만 있음): GH, 경기용
다른 키워드 (키워드_추출okt에만 있음): 용인, 아이디어, 경기, 공모전


In [326]:
print_different_keywords(30)

제목: 용인시 “여성의 경제 참여 증진과 성평등 도시 구현에 힘 모아 달라”
키워드: 용인시,여성,증진,경제,참여,도시,백군기,용인,시장,경기도여성단체협의회
키워드_추출okt: 용인시, 여성, 경제, 참여, 증진, 평등, 도시, 구현, 힘, 달라
다른 키워드 (키워드에만 있음): 경기도여성단체협의회, 용인, 시장, 백군기
다른 키워드 (키워드_추출okt에만 있음): 힘, 달라, 평등, 구현


In [328]:
print_different_keywords(50)

제목: 용인시의회 김상수 의원, '무장애 도시 조성 조례안' 가결
키워드: 의원,용인시의회,김상수,도시,조성,조례안,가결
키워드_추출okt: 용인시, 의회, 김상수, 의원, 무장애, 도시, 조성, 조례, 안, 가결
다른 키워드 (키워드에만 있음): 용인시의회, 조례안
다른 키워드 (키워드_추출okt에만 있음): 안, 의회, 조례, 무장애, 용인시


현재로서는 제목을 이용한 키워드 분석하기 좀 어려움..  
제목에서 키워드 추출을 잘할수만 있다면.. 좀 살릴수 있을거 같은데..

.